# Binary evolution between a star and a neutron star

In this notebook we show the features present in the evolution of a possible progenitor of J08408 from the core collapse and formation of the NS onwards

The evolution is taken from the grid of natal kicks explored and it is labeled as: `natal_kick_id = 309`

### Config stuff

In [ ]:
save_figures = False

In [ ]:
%load_ext autoreload
%autoreload 2

# ---------------------------------------------
# Libraries used
import os
from pathlib import Path
import sys
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FixedLocator, MultipleLocator
from matplotlib.collections import LineCollection
from matplotlib import colors
import pandas as pd
from scipy.optimize import bisect

from mesa_reader import MesaInfo
import kipplotter.mkipp as mkipp
import poskiorb

# ---------------------------------------------
# Set some global options
np.seterr(all="ignore")

# ---------------------------------------------
# Some constants
standard_cgrav = 6.67430e-8  # gravitational constant (g^-1 cm^3 s^-2)
mu_sun = 1.3271244e26
Msun = mu_sun / standard_cgrav
Rsun = 6.957e10  # solar radius (cm)
secyer = 24 * 60 * 60 * 365.25e0  # secods per year
clight = 2.99792458e10 # speed of light in vacuum (cm s^-1)

# ---------------------------------------------
# Load data
root = ".."
run_stst_location = f"{root}/data/raw/star+star"
run_stns_location = f"{root}/data/raw/star+ns"
config_location = f"{root}/config"

# MESA data
star1 = MesaInfo(f"{run_stst_location}/LOGS1/history.data")
star2 = MesaInfo(f"{run_stst_location}/LOGS2/history.data")
companionstar = MesaInfo(f"{run_stns_location}/LOGS_companion/history_309.data")
binary = MesaInfo(f"{run_stns_location}/LOGS_binary/binary_history_309.data")

# core-collapse data fnames
star_cc_fname = f"{run_stst_location}/cc_data/star_at_core_collapse.data"
binary_cc_fname = f"{run_stst_location}/cc_data/binary_at_core_collapse.data"

mpl_style = f"{config_location}/style.mpl"
paper_style = f"{config_location}/paper-style.mpl"

# plot config
MT_cases = ["Case A", "Case B"]
MT_colors = ["#DEBA93", "#D9B6C4"]
mix_zones = ["convection", "semiconvection", "thermohaline", "overshooting"]
mix_colors = ["Chartreuse", "red", "Gold", "purple"]
mix_hatches = ["///", "\\\\\\\\\\\\", "||", "xxx"]
star1_color = "#408191"  # '#D9B6C4'
star2_color = "#3F4364"
companionstar_color = "#3F4364"

### Utility functions

In [ ]:
def group_consecutives(vals, step=1):
    """Return list of consecutive lists of numbers from vals (number list)"""
    run = []
    result = [run]
    expect = None
    for v in vals:
        if (v == expect) or (expect is None):
            run.append(int(v))
        else:
            run = [int(v)]
            result.append(run)
        expect = int(v) + step
    return result


def find_tams(center_h1, model):
    for m in model:
        if center_h1[int(m)] < 1e-5:
            return int(m)

---

# Hertzsprung-Russell diagram

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(3.4, 3.4), nrows=1, ncols=1)

# axis labels
ax.set_xlabel("$\\log\,T_{\\rm eff}/{\\rm K}$")
ax.set_ylabel("$\\log\,L/L_\\odot$")

# constant radii lines
x = np.linspace(0, 8, 100)
for r in (1, 10, 100):
    y = 4 * x + 2 * np.log10(r) - 15.04680947
#    ax.plot(x, y, linestyle=":", color="gray", alpha=0.6, zorder=-99)
#ax.annotate("$1 \\; R_\\odot$", xy=(4.96, 4.73), color="gray", rotation=290)
#ax.annotate("$10 \\; R_\\odot$", xy=(4.465, 4.73), color="gray", rotation=290)

# ZAMS line for Z=0.02
# _, lt, ll = np.loadtxt(fname='zams_hr.data', unpack=True)
# ax.plot(lt, ll, linestyle='--', lw=1, color='black', zorder=-99)

# primary
logTeff = star1.data.log_Teff
logL = star1.data.log_L

ax.plot(logTeff[100:], logL[100:], color=star1_color, label="primary (25 M$_\\odot$)")

# secondary
logTeff = star2.data.log_Teff
logL = star2.data.log_L

ax.plot(logTeff[100:], logL[100:], color=star2_color, label="secondary (22 M$_\\odot$)")

# secondary post collapse of primary into a NS
logTeff = companionstar.data.log_Teff
logL = companionstar.data.log_L
ax.plot(logTeff, logL, color=companionstar_color)


# legends of MT phases
def add_handle(color):
    return patches.Patch(
        fill=True, facecolor=color, edgecolor=color, linewidth=0, alpha=1
    )


handles, legends = [], []
for i, case in enumerate(MT_cases):
    legends.append(case)
    handles.append(add_handle(MT_colors[i]))
fig.legend(
    handles,
    legends,
    handlelength=1.8,
    title="MT phases",
    title_fontsize=9,
    fancybox=True,
    frameon=True,
    facecolor="lightgray",
    edgecolor="dimgray",
    framealpha=1,
    loc="center",
    ncol=1,
    bbox_to_anchor=(0.18, 0.15),
    bbox_transform=ax.transAxes,
)

ax.legend(loc="upper center", ncol=1, columnspacing=0.6, bbox_to_anchor=(0.27, 0.96))

# axis limits & ticks
ax.set_xticks(np.arange(4.3, 5.4, 0.2))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))
#ax.set_xlim(5.25, 4.35)
ax.set_yticks(np.arange(4.6, 6.2, 0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
#ax.set_ylim(4.7, 5.9)

# initial orbital period
# ax.annotate('$P_{\\rm orb} = 3$ days', xy=(4.65, 5.72),
#             bbox=dict(facecolor='lightgray', edgecolor='dimgray', boxstyle='round, pad=0.4'));

ax.invert_xaxis()

plt.subplots_adjust(left=0.15)
# if save_figures: plt.savefig('hr.png', dpi=250);

After 5.24 Myr, the initially more massive star overfills its Roche lobe and starts to donate mass to the companion. This mass-transfer (MT) phase is the so-called Case A of MT.

# Mass transfer & mass accretion

In [ ]:
9# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, axs = plt.subplots(figsize=(4.2, 4.2), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# axis labels
axs[0].set_ylabel("$f_{\\rm mt} = - \\dot{M}_{\\rm NS} / \\dot{M}_{\\rm comp}$")
axs[1].set_ylabel("$\\log\,\\mid\\dot{M}\\mid$ [M$_\\odot$ yr$^{-1}$]")
axs[1].set_xlabel("$M_{\\rm comp}$ [M$_\\odot$]")

# axis limits & ticks
axs[0].set_yticks(np.arange(0.0, 0.2, 0.01))
axs[0].set_ylim([-0.0025, 0.04])
axs[1].set_yticks(np.arange(-8, 0, 1))
axs[1].set_ylim([-8.5, -0.5])
axs[1].set_xticks(np.arange(31, 33, 0.1))
axs[1].set_xlim([32.65, 32.05])

# data
masscompanion = binary.data.star_1_mass
mdot_companion = np.power(10, binary.data.lg_mstar_dot_1)
mdot_ns = np.power(10, binary.data.lg_mstar_dot_2)
mdot_transfer = np.power(10, binary.data.lg_mtransfer_rate)
mdot_companion_wind = np.power(10, binary.data.lg_wind_mdot_1)
mdot_edd_ns = 1e-8 * 1.66e0

# plot
axs[0].plot(masscompanion, binary.data.eff_xfer_fraction, color="black")

axs[1].plot(masscompanion, np.log10(mdot_companion), color=star1_color, label="$\\dot{M}_{\\rm tot, comp}$", zorder=9)
axs[1].plot(masscompanion, np.log10(mdot_transfer), color="black", ls=":", label="$\\dot{M}_{\\rm ROLF, comp}$", zorder=10)
axs[1].plot(masscompanion, np.log10(mdot_companion_wind), color=star1_color, ls=":", label="$\\dot{M}_{\\rm wind, comp}$", zorder=10)
axs[1].plot(masscompanion, 1.09*np.log10(mdot_ns), color=star2_color, label="$\\dot{M}_{\\rm NS} (= \\dot{M}_{\\rm Edd, NS})$", zorder=9)
# axs[1].axhline(y=np.log10(mdot_edd_ns), color="black", ls="--", zorder=10)

axs[1].legend();

At around 18 M$_\odot$ and during the Case A of mass-transfer (MT), the accretor stops to accrete all the material
that reaches its surface, that is why we see a drop in the accretion efficiency ($f_{\rm mt}$)

This same sudden drop in $f_{\rm mt}$ is obtained for the Case B of MT, around 11 M$_\odot$ into the donor star.
In this case, the value of $f_{\rm mt}$ is pretty noisy perhaps due to numerical issues in this faster MT episode

---

## Binary properties during MT phases

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, axs = plt.subplots(figsize=(4.2, 3.2), nrows=3, sharex=True)
plt.subplots_adjust(hspace=0)

# axis labels
axs[0].set_ylabel("$P_{\\rm orb}$ [days]")
axs[1].set_ylabel("$e$")
axs[2].set_ylabel("$P_{\\rm rot, comp} / P_{\\rm orb}$")
axs[2].set_xlabel("$M_{\\rm comp}$ [M$_\\odot$]")

# axis limits & ticks
axs[1].set_xticks(np.arange(31, 33, 0.1))
axs[1].set_xlim(32.65, 32.05)
axs[1].set_ylim(0.45,0.75)

# data
masscompanion = binary.data.star_1_mass
period = binary.data.period_days
eccentricity = binary.data.eccentricity
prot_div_porb = binary.data.P_rot_div_P_orb_1

axs[0].plot(masscompanion, period, color="black")
axs[1].plot(masscompanion, eccentricity, color="black")
axs[2].plot(masscompanion, prot_div_porb, color="black")
axs[2].axhline(y=1, ls=":", color="black");

axs[0].fill_between(x=[axs[0].get_xlim()[0], axs[0].get_xlim()[1]], y1=9.54-1.0, y2=9.54+1.0)
axs[1].fill_between(x=[axs[1].get_xlim()[0], axs[1].get_xlim()[1]], y1=0.63-0.1, y2=0.63+0.1);

---

## Donor properties during MT phases

Evolution of rotation rate, luminosity and effective temperature due to mass accretion

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, axs = plt.subplots(figsize=(4.2, 3.2), nrows=3, sharex=True)
plt.subplots_adjust(hspace=0)

# inset plot
# left, bottom, width, height = [0.47, 0.72, 0.15, 0.15]
# ax2 = fig.add_axes([left, bottom, width, height])

# axis labels
axs[0].set_ylabel("$\\Omega / \\Omega_{\\rm crit}$")
axs[1].set_ylabel("$\\log\\,(L/L_\\odot)$")
latex = "{\\rm eff}"
axs[2].set_ylabel(f"$\\log\\,(T_{latex}/K)$")
axs[2].set_xlabel("$M_1$ [M$_\\odot$]")

# data
masscompanion = binary.data.star_1_mass
# omega_div_omega_crit_1A = star2.data.surf_avg_omega_div_omega_crit[numbers[0]]
omega_div_omega_crit_companion = companionstar.data.surf_avg_omega_div_omega_crit
log_Teff_companion = companionstar.data.log_Teff
log_L_companion = companionstar.data.log_L
log_Lnuc_companion = companionstar.data.log_Lnuc

# plot
axs[0].plot(masscompanion, omega_div_omega_crit_companion, color=star2_color)
# ax2.plot(mass_1B, omega_div_omega_crit_1B, color=star2color, lw=0.5)

axs[1].plot(masscompanion, log_L_companion, color=star2_color, label="$\\log\,L$")
axs[1].plot(masscompanion, log_Lnuc_companion, color="black", ls="--", label="$\\log\,L_{\\rm nuc}$")

axs[2].plot(masscompanion, log_Teff_companion, color=star2_color)

# axis limits & ticks
axs[0].set_xticks(np.arange(31, 33, 0.1))
axs[0].xaxis.set_minor_locator(MultipleLocator(0.05))
axs[0].set_xlim([32.65, 32.05])

# axs[0].set_yticks(np.arange(0, 1.35, 0.25))
# axs[0].yaxis.set_minor_locator(MultipleLocator(0.125))
# axs[0].set_ylim(0.125, 1.125)

# axs[1].set_yticks(np.arange(3, 6, 0.2))
# axs[1].yaxis.set_minor_locator(MultipleLocator(0.1))
# axs[1].set_ylim(4.9, 5.7)

# axs[2].set_yticks(np.arange(4, 5, 0.05))
# axs[2].yaxis.set_minor_locator(MultipleLocator(0.025))
# axs[2].set_ylim(4.425, 4.575)

# ax2.set_xticks(np.arange(8.8, 9.3, 0.05))
# ax2.xaxis.set_minor_locator(MultipleLocator(0.025))
# ax2.set_xlim(9.22, 9.025)  # 9.12,8.98)
# ax2.set_yticks(np.arange(0.8, 1.1, 0.05))
# ax2.yaxis.set_minor_locator(MultipleLocator(0.025))
# ax2.set_ylim(0.775, 0.995)  # 0.975)#0.775, 0.97);

axs[1].legend(loc=6, ncol=1);